In [1]:
import numpy as np
import torch
import mmengine
from mmdet3d.structures import LiDARInstance3DBoxes

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-vt8syj77 because the default path (/home/hello/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [15]:
# info_path = '../data/nuscenes/nuscenes_infos_train.pkl'
info_path = '../data/CODA/coda_infos_val.pkl'
# info_path = '../test/work_dirs/detection_results.pkl'
info = mmengine.load(info_path)

In [12]:
info.keys()

dict_keys(['sample_idx', 'pre_bboxes', 'pre_labels', 'pre_scores'])

In [14]:
info['pre_bboxes'][10]

array([[ 8.7822247e+00, -1.1105194e+00, -4.5581961e-01,  1.0127645e+00,
         7.7149779e-01,  1.6541657e+00,  2.7920138e-02],
       [ 4.0178108e+00,  1.3444443e+00, -7.2432631e-01,  1.2494777e+00,
         1.0411749e+00,  1.8735346e+00, -3.1302905e+00],
       [-1.3226414e+00, -6.5435410e-02, -8.8374972e-01,  6.8995965e-01,
         7.0420927e-01,  1.6291530e+00, -4.2840250e-02],
       [-1.0327726e+01,  1.9955215e+00, -1.1654058e+00,  9.9925840e-01,
         7.8190684e-01,  1.6514391e+00, -3.0467377e+00],
       [ 6.8510590e+00, -8.5642395e+00, -8.2799816e-01,  1.2560598e+00,
         1.0953188e+00,  1.8705397e+00, -2.7527504e+00],
       [ 1.6808777e+01, -1.0247383e+00, -1.4880478e-01,  1.1772968e+00,
         9.6578699e-01,  1.9188216e+00, -1.8803013e-02],
       [-1.8951054e+01,  2.3980293e+00, -1.5141811e+00,  1.0095267e+00,
         8.0558598e-01,  1.7133368e+00, -3.1023571e+00],
       [-3.1954002e-01,  5.6252480e-01, -8.3895195e-01,  6.6365540e-01,
         6.9590956e-01,  

In [ ]:
info['metainfo']

In [10]:
info['data_list'][10]

{'scene': '16',
 'frame': 3284,
 'sample_idx': 10,
 'token': '16_3284',
 'timestamp': '1675881597.623372',
 'lidar_points': {'num_pts_feats': 4,
  'lidar_path': '3d_comp/os1/16/3d_comp_os1_16_3284.bin'},
 'images': {'cam0': {'img_path': '2d_rect/cam0/16/2d_rect_cam0_16_3284.png',
   'cam2img': array([[765.90139253,   0.        , 631.19869232],
          [  0.        , 765.90139253, 525.73421478],
          [  0.        ,   0.        ,   1.        ]]),
   'lidar2cam': array([[-0.02018827, -0.9997873 ,  0.00421773,  0.08023904],
          [-0.13130881, -0.00153064, -0.99134033, -0.09986692],
          [ 0.99113593, -0.02056727, -0.13124998, -0.02980426],
          [ 0.        ,  0.        ,  0.        ,  1.        ]]),
   'timestamp': '1675881597.623372'},
  'cam1': {'img_path': '2d_rect/cam1/16/2d_rect_cam1_16_3284.png',
   'cam2img': array([[765.90139253,   0.        , 631.19869232],
          [  0.        , 765.90139253, 525.73421478],
          [  0.        ,   0.        ,   1.      

In [16]:
info['data_list'][10]['instances']

[{'bbox_label_3d': 1,
  'bbox_3d': [-10.310960463113979,
   2.028716641360134,
   -0.36027068188413436,
   0.865833800091182,
   0.7841317598317799,
   1.5720882879954945,
   -3.1106214688987],
  'is_occluded': 'Unknown',
  'instance_id': 'Pedestrian:6'},
 {'bbox_label_3d': 1,
  'bbox_3d': [3.996957694587053,
   1.301308912428894,
   0.29934823747993544,
   1.1663049860561538,
   1.1610492506270165,
   1.7803059047734597,
   3.11089167934125],
  'is_occluded': 'None',
  'instance_id': 'Pedestrian:8'},
 {'bbox_label_3d': 1,
  'bbox_3d': [6.293376322275947,
   -9.764235622546977,
   -0.08548870668069863,
   1.1473814547873364,
   0.7446483176668267,
   1.3123044339059788,
   -3.135643076743542],
  'is_occluded': 'Unknown',
  'instance_id': 'Pedestrian:9'},
 {'bbox_label_3d': 1,
  'bbox_3d': [6.834217870755204,
   -8.767150305543638,
   0.19027036380026185,
   1.3612321374235963,
   1.1147405872447391,
   1.888957291625827,
   3.019946741946961],
  'is_occluded': 'Unknown',
  'instance_id

In [9]:
info['data_list'][-1]['ego2global']

array([[ 3.59959949e-02,  9.99326181e-01,  7.17445643e-03,
        -5.94616790e+01],
       [-9.98627400e-01,  3.62422942e-02, -3.78128506e-02,
         9.20977940e+01],
       [-3.80473904e-02, -5.80349759e-03,  9.99259083e-01,
        -2.22156000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [33]:
len(info['data_list'])

669

In [ ]:
instance_range = [np.inf, np.inf, np.inf, -np.inf, -np.inf, -np.inf]
corners = []
for sample in info['data_list']:
    bboxes = [instance['bbox_3d'] for instance in sample['instances']]
    bboxes_3d = LiDARInstance3DBoxes(bboxes, origin=(0.5, 0.5, 0.5))
    corner = bboxes_3d.corners.reshape(-1, 3).numpy()
    corners.append(np.concatenate([corner.min(axis=0), corner.max(axis=0)]))
    instance_range[0:3] = np.minimum(instance_range[0:3], np.min(corner, axis=0))
    instance_range[3:6] = np.maximum(instance_range[3:6], np.max(corner, axis=0))

In [35]:
sum = 0
distribute = [0, 0, 0, 0, 0, 0, 0, 0, 0]
for sample in info['data_list']:    
    for instance in sample['instances']:
        if instance['bbox_label_3d'] == 2:
            sum += 1
            if instance['bbox_3d'][5] < 1.0:
                distribute[0] += 1
            elif instance['bbox_3d'][5] < 1.1:
                distribute[1] += 1
            elif instance['bbox_3d'][5] < 1.2:
                distribute[2] += 1
            elif instance['bbox_3d'][5] < 1.3:
                distribute[3] += 1
            elif instance['bbox_3d'][5] < 1.4:
                distribute[4] += 1
            elif instance['bbox_3d'][5] < 1.5:
                distribute[5] += 1
                print(instance['bbox_3d'], sample['token'])
            elif instance['bbox_3d'][5] < 1.6:
                distribute[6] += 1
            elif instance['bbox_3d'][5] < 1.7:
                distribute[7] += 1
            else:
                distribute[8] += 1
print(sum, distribute)

[1.2897008136073254, -14.807167961677425, 0.7826085844058754, 1.5710024460546472, 0.5062104877818576, 1.4985126459604317, -2.5560207238018675] 11_1724
[0.5440399531439226, -14.754028340541325, 0.9222201899312568, 1.5710024460546472, 0.5062104877818576, 1.4985126459604317, -2.5575161200630796] 11_1732
[-7.50805003011844, -14.776964601089539, 0.5612125550295457, 1.5710024460546472, 0.5062104877818576, 1.4985126459604317, -2.525803124517707] 11_1821
[-9.900794527328486, -14.94275768909371, 0.46515919315427645, 1.5710024460546472, 0.5062104877818576, 1.4985126459604317, -2.5191553502491626] 11_1845
[-11.301879007638831, -14.938330604309307, 0.33012557849707447, 1.5710024460546472, 0.5062104877818576, 1.4985126459604317, -2.515378948595139] 11_1860
1034 [203, 468, 179, 77, 8, 5, 26, 26, 42]


In [ ]:
corners

In [ ]:
instance_range

In [ ]:
info['data_list'][10]['instances']

In [ ]:
info['data_list'][0]['images']['cam1']

In [ ]:
BBOX_CLASS_TO_ID = {
    # Dynamic Classes
    "Car"                   : 0,
    "Pedestrian"            : 1,
    "Bike"                  : 2,
    "Motorcycle"            : 3,
    "Golf Cart"             : 4, # Unused
    "Truck"                 : 5, # Unused
    "Scooter"               : 6,
    # Static Classes
    "Tree"                  : 7,
    "Traffic Sign"          : 8,
    "Canopy"                : 9,
    "Traffic Light"         : 10,
    "Bike Rack"             : 11,
    "Bollard"               : 12,
    "Construction Barrier"  : 13, # Unused
    "Parking Kiosk"         : 14,
    "Mailbox"               : 15,
    "Fire Hydrant"          : 16,
    # Static Class Mixed
    "Freestanding Plant"    : 17,
    "Pole"                  : 18,
    "Informational Sign"    : 19,
    "Door"                  : 20,
    "Fence"                 : 21,
    "Railing"               : 22,
    "Cone"                  : 23,
    "Chair"                 : 24,
    "Bench"                 : 25,
    "Table"                 : 26,
    "Trash Can"             : 27,
    "Newspaper Dispenser"   : 28,
    # Static Classes Indoor
    "Room Label"            : 29,
    "Stanchion"             : 30,
    "Sanitizer Dispenser"   : 31,
    "Condiment Dispenser"   : 32,
    "Vending Machine"       : 33,
    "Emergency Aid Kit"     : 34,
    "Fire Extinguisher"     : 35,
    "Computer"              : 36,
    "Television"            : 37, # unused
    "Other"                 : 38,
    "Horse"                 : 39,
    # New Classes
    "Pickup Truck"          : 40,
    "Delivery Truck"        : 41,
    "Service Vehicle"       : 42,
    "Utility Vehicle"       : 43,
    "Fire Alarm"            : 44,
    "ATM"                   : 45,
    "Cart"                  : 46,
    "Couch"                 : 47,
    "Traffic Arm"           : 48,
    "Wall Sign"             : 49,
    "Floor Sign"            : 50,
    "Door Switch"           : 51,
    "Emergency Phone"       : 52,
    "Dumpster"              : 53,
    "Vacuum Cleaner"        : 54, # unused
    "Segway"                : 55,
    "Bus"                   : 56,
    "Skateboard"            : 57,
    "Water Fountain"        : 58
}

In [ ]:
METAINFO = {
        'classes':
        tuple(BBOX_CLASS_TO_ID.keys()),
        'version':
        'v1.0-trainval',
        'palette': [
            (255, 158, 0),  # Orange
            (255, 99, 71),  # Tomato
            (255, 140, 0),  # Darkorange
            (255, 127, 80),  # Coral
            (233, 150, 70),  # Darksalmon
            (220, 20, 60),  # Crimson
            (255, 61, 99),  # Red
            (0, 0, 230),  # Blue
            (47, 79, 79),  # Darkslategrey
            (112, 128, 144),  # Slategrey
        ]
    }

In [ ]:
METAINFO['classes']